In [1]:
# Initializing Spark
with open("setupPySpark.py", "r") as setup_file:
    exec(setup_file.read())


In [2]:
# Spark context
from pyspark.sql.session import SparkSession

spark = SparkSession(sc)

In [3]:
# SQL context
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

In [4]:
# Importing required functions
from util import read_file, read_folder, get_character_similarity
from pandas import Series, DataFrame
from util_spark import remove_stopwords_spark, detect_language_spark, flatten_list_of_tokens, spell_correct_tokens_spark, get_semantic_similarity_spark
from tokenization_spark import tokenize_sentence_nltk_spark
from pyspark.sql.functions import col
from pos_tagging_spark import run_treetagger_pos_tag_spark
from modeling_spark import run_word2vec_model_pyspark

In [5]:
# Reading input file(s) using python's default libraries
in_file = open("in_file.cfg").read()
in_file = in_file.split("\n")
patterns_file = in_file[5]
file_folder = in_file[4]
label = in_file[3]
column = in_file[2]
in_type = in_file[1]
in_file = in_file[0]
if file_folder == "file":
    strings = read_file(in_file, in_type = in_type)
    if in_type == "text":
        strings = tokenize_sentence_nltk(strings)
        strings = DataFrame(strings)[0]
    elif in_type == "html":
        timestamp = strings[2]
        meta_data = strings[1]
        strings = strings[0]
        strings[label] = meta_data["Comment"]
        labels = strings[label]
        strings = strings[column]
    else:
        if label in strings.columns:
            labels = strings[label]
        strings = strings[column]
else:
    strings = read_folder(in_file, in_type = in_type)
    patterns = Series([".*" + x + ".*" for x in open(patterns_file, 'r').readlines()])

In [6]:
# Appending conversation together and creating spark data frome
strings['conversation'] = strings['conversation'].apply(lambda x: ". ".join(x["Message"]))
sentenceDataFrame = spark.createDataFrame(strings)

In [7]:
# Creating list of sentences for each conversation
sentenceDataFrame = tokenize_sentence_nltk_spark(df = sentenceDataFrame, in_col = "conversation")

In [8]:
# Language identification and filtering
sentenceDataFrame = detect_language_spark(df = sentenceDataFrame, in_col = "conversation", out_col = "language")
sentenceDataFrame = sentenceDataFrame.where(col('language') == "en")

In [9]:
# POS tagging and lemmatization using TreeTagger
sentenceDataFrame = run_treetagger_pos_tag_spark(df = sentenceDataFrame, in_col = "conversation", out_col = "pos", get_lemma = True)

In [10]:
# Merging 2 consecutive words if a) Words are incorrectly spelled and b) Merged word is correctly spelled
sentenceDataFrame = spell_correct_tokens_spark(df = sentenceDataFrame, in_col = "pos")

In [11]:
# Flattening out token of rows and running word2vec model
sentenceDataFrame = flatten_list_of_tokens(sentenceDataFrame, in_col = "pos")
model, sentenceDataFrame = run_word2vec_model_pyspark(sentenceDataFrame, in_col = "pos", vec_size = 100, in_type = "tokens", out_col = "result")

In [12]:
# Collecting document vectors in a list
doc_vecs = []
for row in sentenceDataFrame.select('result').collect():
    doc_vecs = doc_vecs + [row['result']]

In [13]:
sim1 = get_semantic_similarity_spark(model)

In [14]:
sim1.head()

,for,this,in,move,have,your,point,guy,b2b,agree,...,no,we,let,ramification,when,away,and,today,the,cant
for,1.000000,0.134573,-0.178252,0.074515,0.011317,0.030545,0.050908,-0.206047,-0.061516,-0.148625,...,-0.005031,0.008286,0.043558,0.012839,0.004365,-0.161527,0.059483,-0.019999,-0.271658,0.024824
this,0.134573,1.000000,0.084741,0.001460,0.135461,-0.132793,-0.073039,-0.160185,0.032266,0.035617,...,0.166186,0.107087,-0.051598,0.233857,-0.036789,0.041292,0.098604,0.052540,0.166813,0.042530
in,-0.178252,0.084741,1.000000,0.089618,-0.115539,-0.039701,-0.002061,0.002920,-0.056720,-0.122512,...,-0.049752,0.118345,0.169684,-0.031200,-0.016943,-0.090282,0.162128,0.025246,0.099597,-0.030526
move,0.074515,0.001460,0.089618,1.000000,0.158001,-0.137687,0.183042,-0.073193,-0.072524,0.178970,...,0.099382,0.038360,0.010177,-0.018346,0.095216,-0.085638,-0.079254,0.011398,-0.000120,0.138376
have,0.011317,0.135461,-0.115539,0.158001,1.000000,-0.286646,0.037367,-0.053461,-0.070606,0.137010,...,-0.114118,-0.009014,-0.059815,-0.039275,-0.034229,-0.016343,-0.067755,-0.103216,0.153610,-0.074666


In [15]:
ratio = get_character_similarity(sim1.columns, ratio_type = 'ratio')

In [16]:
partial_ratio = get_character_similarity(sim1.columns, ratio_type = 'partial_ratio')

In [17]:
token_sort_ratio = get_character_similarity(sim1.columns, ratio_type = 'token_sort_ratio')

In [18]:
token_set_ratio = get_character_similarity(sim1.columns, ratio_type = 'token_set_ratio')

In [19]:
sim1 = sim1[ratio.columns]
sim1 = sim1.loc[ratio.columns]

In [20]:
semantic_weight = 0.5
ratio_weight = 0.4
partial_ratio_weight = 0.4
token_sort_ratio_weight = 0.1
sim = 1 - (semantic_weight*sim1 + (ratio_weight*ratio + partial_ratio_weight*partial_ratio + token_sort_ratio_weight*token_sort_ratio + (1-ratio_weight-partial_ratio_weight-token_sort_ratio_weight)*token_set_ratio)*(1-semantic_weight))